In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
# sys.path.append("../utils")
sys.path.append("../../../stimuli")

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
from functools import reduce
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# import drawing_utils as drawing
# import importlib
# import scoring

sys.path.append("../../stimuli/towers/block_utils/")
import blockworld_utils as utils

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

results_csv_directory = "../../results/csv"

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [49]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'../../auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

db = conn['lax']

In [50]:
subdomains = {
    'structures' :  ['bridge', 'castle', 'house', 'city'],
    'drawing' :  ['nuts-bolts','wheels','furniture']
}

domains = list(all_subdomains.keys())

In [72]:
# manually specify
domain = 'structures'
subdomain = 'bridge'

In [73]:
col_name = 'lax_{}_corpus'.format(domain)
iteration_name = 'corpus_dev'
col_name

'lax_structures_corpus'

In [74]:
coll = db[col_name]

In [67]:
# really_run = False;

# if really_run:
#     db.drop_collection(col_name)

In [35]:
# # Find people who have completed all trials
# query = coll.find({"$and":[
#                         {'trialType':'normal-trial'},
#                         {'eventType':'trialEnd'},
#                         {'iterationName': { '$in': [iteration_name] }},
#                         {'trialNum': numTrials-1}]
#                      })

# complete_data_df = pd.DataFrame(query.sort('absoluteTime'))
# complete_data_ids = list(complete_data_df['gameID'])
# print(str(len(complete_data_ids)) + ' participants found')


# pd.DataFrame(coll.find({'iterationName': iteration_name}))

In [75]:
df_all = pd.DataFrame(coll.find({"$and":[ {'iterationName': iteration_name},
#                                           {'experimentName': experiment_name}
                                         ]}))

# df_all['experimentName']

In [76]:
# assign domain and subdomain

# check prefix of 'lax' for all config names

assert(np.all(df_all['experimentName'].apply(lambda x: x.split('-')[0]) == 'lax'))
assert(np.all(df_all['experimentName'].apply(lambda x: x.split('-')[1] in domains)))
assert(np.all(df_all['experimentName'].apply(lambda x: x.split('-')[2] in subdomains[domain])))

df_all['domain'] = df_all['experimentName'].apply(lambda x: x.split('-')[1])
df_all['subdomain'] = df_all['experimentName'].apply(lambda x: x.split('-')[2])

In [78]:
df_all.columns

Index(['_id', 'partitionFamily', 'splitNumber', 'stimIDs', 'stimURLS',
       'ntrials', 'stimGroups', 'numGames', 'games', 'experimentType',
       'experimentName', 's3_bucket_url', 'versionInd', 'datatype',
       'iterationName', 'condition', 'config_name', 'gameID', 'shuffle',
       'trialOrder', 'view_history', 'rt', 'trial_type', 'trial_index',
       'time_elapsed', 'internal_node_id', 'responses', 'stimulus', 'stimId',
       'stimURL', 'trialDomain', 'domain', 'subdomain'],
      dtype='object')

In [93]:
df_all[['iterationName', 'condition', 'config_name', 'gameID', 'shuffle',
       'trialOrder', 'view_history', 'rt', 'trial_type', 'trial_index',
       'time_elapsed', 'internal_node_id', 'responses', 'stimulus', 'stimId',
       'stimURL', 'trialDomain', 'domain', 'subdomain']]

,iterationName,condition,config_name,gameID,shuffle,trialOrder,view_history,rt,trial_type,trial_index,time_elapsed,internal_node_id,responses,stimulus,stimId,stimURL,trialDomain,domain,subdomain
0,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,True,"[194, 240, 196, 187, 019, 009, 144, 017, 103, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,structures,bridge
1,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,"[{""page_index"":0,""viewing_time"":1546.700000047...",3167.9,instructions,0.0,3170.0,0.0-0.0,NaN,NaN,NaN,NaN,NaN,structures,bridge
2,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,10776.3,jspsych-category-labels-display,1.0,13958.0,0.0-1.0,"{""what-1"":""a"",""where-1"":""sfg"",""what-2"":""wrga"",...","""https://lax-structures-bridge-all.s3.amazonaw...",229,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
3,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,6641.2,jspsych-category-labels-display,2.0,21114.0,0.0-2.0,"{""what-1"":""wefaw"",""where-1"":""awefwef"",""what-2""...","""https://lax-structures-bridge-all.s3.amazonaw...",065,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
4,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,2968.7,jspsych-category-labels-display,3.0,24594.0,0.0-3.0,"{""what-1"":""ef"",""where-1"":""fe""}","""https://lax-structures-bridge-all.s3.amazonaw...",064,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
5,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,2313.3,jspsych-category-labels-display,4.0,27422.0,0.0-4.0,"{""what-1"":""wef"",""where-1"":""fe""}","""https://lax-structures-bridge-all.s3.amazonaw...",196,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
6,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,2973.3,jspsych-category-labels-display,5.0,30909.0,0.0-5.0,"{""what-1"":""esfefse"",""where-1"":""sefesf""}","""https://lax-structures-bridge-all.s3.amazonaw...",194,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
7,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,10249.9,jspsych-category-labels-display,6.0,41671.0,0.0-6.0,"{""what-1"":""sefsef"",""where-1"":""sfesf"",""what-2"":...","""https://lax-structures-bridge-all.s3.amazonaw...",139,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
8,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,4326.9,jspsych-category-labels-display,7.0,46514.0,0.0-7.0,"{""what-1"":""rgwg3"",""where-1"":""w3gw3g43g""}","""https://lax-structures-bridge-all.s3.amazonaw...",187,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge
9,corpus_dev,all,lax-structures-bridge-corpus-procedural,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,NaN,NaN,NaN,2344.3,jspsych-category-labels-display,8.0,49370.0,0.0-8.0,"{""what-1"":""rgwr"",""where-1"":""srg""}","""https://lax-structures-bridge-all.s3.amazonaw...",239,https://lax-structures-bridge-all.s3.amazonaws...,towers,structures,bridge


## Filter out metadata

Info about stimulus set downloaded from mongo.
Contains info about tower type.

In [89]:
df_meta = df_all[(df_all.datatype == 'stim_metadata') &
            (df_all.iterationName == iteration_name)]\
        [["gameID","partitionFamily","splitNumber","stimIDs", "stimURLS", "stimGroups",
          "numGames","experimentType","experimentName","versionInd"]]

In [90]:
df_meta

,gameID,partitionFamily,splitNumber,stimIDs,stimURLS,stimGroups,numGames,experimentType,experimentName,versionInd
0,8469-a75da4fa-681f-425e-9766-a00b3d52e2b6,0.0,1.0,"[009, 017, 019, 039, 041, 050, 064, 065, 096, ...",[https://lax-structures-bridge-all.s3.amazonaw...,"{'103': 'test', '115': 'test', '139': 'train',...",0.0,corpus,lax-structures-bridge-corpus-procedural,0.0


## Grab trial data

In [105]:
df_trial = df_all[(df_all.datatype == 'trial_end') &
             (~pd.isna(df_all.stimId)) & #has stim ID
             (df_all.iterationName == iteration_name)]\
            [['datatype', 'iterationName', 'condition', 'domain', 'subdomain',
       'config_name', 'gameID', 'shuffle', 'trialOrder', 'rt', #'workerID', 
       'trial_type', 'trial_index', 'time_elapsed', 'internal_node_id',
       'view_history', 'stimId', 'stimURL', 'responses']]

### Add metadata to trial data

In [106]:
# verify stim groups in metadata are correct
dicts = list(df_all[df_all.datatype=='stim_metadata']['stimGroups'])

stim_groups = reduce(lambda dict1, dict2: {**dict1, **dict2}, dicts)
pd.Series(list(stim_groups.values())).value_counts()

# assign stim groups from metadata
df_trial['stim_group'] = df_trial['stimId'].apply(lambda stim: stim_groups[stim])
df_trial = df_trial.merge(df_meta, how='left', on='gameID')

## Todo: get complete data sets

In [107]:
# find full datasets

expected_trials = 25

did_complete = df_trial.groupby(['gameID']).count()['datatype'] == expected_trials
print('complete:', len(list(did_complete[did_complete].index)))
print('incomplete:', len(list(did_complete[~did_complete].index)))

complete_dataset_gameIDs = list(did_complete[did_complete].index)

df_trial = df_trial[df_trial.gameID.isin(complete_dataset_gameIDs)]

complete: 1
incomplete: 0


## Data preprocessing

In [108]:
df_trial['labeling_task'] = df_trial['experimentName'].apply(lambda x: x.split('_')[-1])

#### splitting responses

In [109]:
# if experiment_condition == procedural: #check if prodedural (i.e. structured answers)

In [110]:
# df_trial['responses'] = df_trial['responses'].apply(ast.literal_eval)

In [111]:
# responses = df_trial['responses'][0]
# for k in responses.keys():
#     q, row = k.split('-')

## Save out data

In [118]:
really_save = True

if really_save:
    df_trial.to_csv(results_csv_directory + '/lax-' + domain + '-corpus-' + iteration_name + '.csv')

In [121]:
'/lax-' + domain + '-corpus-' + iteration_name + '.csv'

'/lax-structures-corpus-corpus_dev.csv'

In [48]:
# Search for a specific Prolific ID

df_all[df_all.workerID == '']

,_id,partitionFamily,splitNumber,stimNumbers,ntrials,stimGroups,numGames,games,experimentType,experimentName,...,trial_type,trial_index,time_elapsed,internal_node_id,view_history,label,stimId,stimURL,responses,extra_metadata_index


In [103]:
# Find out how many times each stimulus partitions has been completed

df_trial.groupby(['partitionFamily','splitNumber']).count() / expected_trials

,,datatype,iterationName,condition,config_name,gameID,shuffle,trialOrder,rt,trial_type,trial_index,...,responses,stim_group,stimIDs,stimURLS,stimGroups,numGames,experimentType,experimentName,versionInd,labeling_task
partitionFamily,splitNumber,,,,,,,,,,,,,,,,,,,,,
0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
